In [181]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import random

In [45]:
!ls

data split.ipynb    final lab notes.rtf int_1per.csv        inter_1pc.csv


In [46]:
data=pd.read_csv('inter_1pc.csv')

In [174]:
data

,user_id,book_id,is_read,rating,is_reviewed
0,431147,414799,1,5,0
1,431147,147207,1,3,0
2,431147,943,1,5,0
3,431147,613,1,5,0
4,431147,944,1,5,0
...,...,...,...,...,...
2436190,29544,227034,0,0,0
2436191,29544,1181994,0,0,0
2436192,29544,522467,0,0,0
2436193,29544,210066,0,0,0


In [188]:
# note that there are some interactions where people read the book but did not give a rating
# although most of the time, if the rating is 0, its more likely because people did not read the book
data[data['rating'] == 0][(data[data['rating'] == 0]['is_read']!= 0)]

,user_id,book_id,is_read,rating,is_reviewed
264,431161,1068318,1,0,1
268,431161,74005,1,0,0
269,431161,261605,1,0,0
283,431161,160933,1,0,1
290,431161,113803,1,0,0
...,...,...,...,...,...
2436093,29544,21277,1,0,0
2436105,29544,391542,1,0,0
2436113,29544,19739,1,0,0
2436159,29544,270395,1,0,0


In [173]:
# a panda series containing only users who have read more than 10 books
more_10 = data.groupby('user_id').count()['is_read'][data.groupby('user_id').count()['is_read'] >= 10]
more_10

user_id
106       384
364       844
442       211
532       261
563       500
         ... 
875806     18
875859     57
875939     28
876021     27
876025     46
Name: is_read, Length: 7838, dtype: int64

In [151]:
list(more_10.index)[:5]

[106, 364, 442, 532, 563]

In [155]:
# only keep those users in our data
data = data[data['user_id'].isin(list(more_10.index))]

In [157]:
# this is a list of all unique user_id's present in the dataframe, should be the same as the length of more_10
users = data['user_id'].unique().tolist()
len(users) 

7838

In [158]:
lst = random.shuffle([1,2,3,4])
print(lst)

None


In [159]:
lst = [1,2,3,4]
random.shuffle(lst)
lst

[4, 1, 2, 3]

In [160]:
random.shuffle(users) # randomly shuffle the user id's
# then, all the sequential partition will be random

train_users = users[0:int(len(users) * 0.6)] # a list of user_id's for train set
val_users = users[int(len(users) * 0.6) : int(len(users) * 0.8)]# a list of user_id's for validation set
test_users = users[int(len(users) * 0.8) : ]# a list of user_id's for validation set

In [161]:
# boolean indexing for membership test
data['user_id'].isin([431147,1])

0           True
1           True
2           True
3           True
4           True
           ...  
2436190    False
2436191    False
2436192    False
2436193    False
2436194    False
Name: user_id, Length: 2432070, dtype: bool

In [162]:
train = data[data['user_id'].isin(train_users)]
val = data[data['user_id'].isin(val_users)]
test = data[data['user_id'].isin(test_users)]

In [163]:
final_train = train

In [164]:
# for each USER in the validation set, move 50% of its interactions back into train, only leaving the other half
final_val = pd.DataFrame()

for user in val_users:
    user_df = val[val['user_id'] == user]
    user_train, user_val = train_test_split(user_df, train_size=0.5)
    final_train = final_train.append(user_train)
    final_val = final_val.append(user_val)

In [165]:
# for each USER in the test set, move 50% of its interactions back into train, only leaving the other half
final_test = pd.DataFrame()

for user in test_users:
    user_df = test[test['user_id'] == user]
    user_train, user_test = train_test_split(user_df, test_size=0.5)
    final_train = final_train.append(user_train)
    final_test = final_test.append(user_test)

In [166]:
len(final_val['user_id'].unique()) # only has 20% of users

1568

In [167]:
len(final_test['user_id'].unique()) # only has 20% of users

1568

In [168]:
len(final_train['user_id'].unique()) # has every user

7838

In [169]:
# save df to csv file for easier later access 

final_val.to_csv('final_val.csv')
final_test.to_csv('final_test.csv')
final_train.to_csv('final_train.csv')

In [170]:
!ls

data split.ipynb    final_test.csv      final_val.csv       inter_1pc.csv
final lab notes.rtf final_train.csv     int_1per.csv
